In [ ]:
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


In [ ]:
%%capture
!pip install pyserini==0.22.0
!pip install faiss-cpu

import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"

In [ ]:
! pip freeze | grep pyserini

pyserini==0.22.0


In [ ]:
def dump_jsonl(data, output_path, append=False):
    """
    Write list of objects to a JSON lines file.
    """
    mode = 'a+' if append else 'w'
    with open(output_path, mode, encoding='utf-8') as f:
        for line in data:
            json_record = json.dumps(line, ensure_ascii=False)
            f.write(json_record + '\n')
    print('Wrote {} records to {}'.format(len(data), output_path))

In [ ]:
import json
import re
from bs4 import BeautifulSoup

with open('/content/drive/MyDrive/makarem.xml', 'r', encoding="utf8") as f:
  content = f.read()

soup= BeautifulSoup(content, 'xml')

documents = []

verses = soup.find_all("s")
for verse in verses:
  doc_info = {}
  doc_info['id'] = verse.get('id')
  doc_info['contents'] = re.sub("[\(\[].*?[\)\]]", "", verse.contents[0])
  documents.append(doc_info)

output_path = '/content/drive/MyDrive/documents.jsonl'
dump_jsonl(documents, output_path)

Wrote 6236 records to /content/drive/MyDrive/documents.jsonl


index construction

In [ ]:
! python -m pyserini.index.lucene \
  --collection JsonCollection \
  --input /content/drive/MyDrive/tests/resources/sample_collection_jsonl \
  --language fa \
  --index /content/drive/MyDrive/indexes/sample_collection_jsonl \
  --generator DefaultLuceneDocumentGenerator \
  --threads 1 \
  --storePositions --storeDocvectors --storeRaw

2023-10-10 05:58:17,129 INFO  [main] index.IndexCollection (IndexCollection.java:380) - Setting log level to INFO
2023-10-10 05:58:17,131 INFO  [main] index.IndexCollection (IndexCollection.java:383) - Starting indexer...
2023-10-10 05:58:17,131 INFO  [main] index.IndexCollection (IndexCollection.java:384) - ============ Loading Parameters ============
2023-10-10 05:58:17,131 INFO  [main] index.IndexCollection (IndexCollection.java:385) - DocumentCollection path: /content/drive/MyDrive/tests/resources/sample_collection_jsonl
2023-10-10 05:58:17,132 INFO  [main] index.IndexCollection (IndexCollection.java:386) - CollectionClass: JsonCollection
2023-10-10 05:58:17,132 INFO  [main] index.IndexCollection (IndexCollection.java:387) - Generator: DefaultLuceneDocumentGenerator
2023-10-10 05:58:17,133 INFO  [main] index.IndexCollection (IndexCollection.java:388) - Threads: 1
2023-10-10 05:58:17,133 INFO  [main] index.IndexCollection (IndexCollection.java:389) - Language: fa
2023-10-10 05:58:17

In [ ]:
from pyserini.search.lucene import LuceneSearcher

searcher = LuceneSearcher('/content/drive/MyDrive/indexes/sample_collection_jsonl')
searcher.set_language('fa')
hits = searcher.search('ابولهب')

for i in range(len(hits)):
    print(f'{i+1:2} {hits[i].docid:4} {hits[i].score:.5f}')

 1 s111.1 4.94470


In [ ]:
! python -m pyserini.search \
    --topics msmarco-passage-dev-subset \
    --index msmarco-v1-passage \
    --output run.msmarco-passage.txt \
    --output-format msmarco \
    --bm25


Using pre-defined topic order for msmarco-passage-dev-subset
Attempting to initialize pre-built index msmarco-v1-passage.
lucene-index.msmarco-v1-passage.20220131.9ea315.tar.gz:  22% 450M/2.03G [00:07<00:20, 81.8MB/s]

In [ ]:
!python -m pyserini.index.lucene \
  --collection JsonCollection \
  --input tests/resources/sample_collection_jsonl \
  --index indexes/sample_collection_jsonl \
  --generator DefaultLuceneDocumentGenerator \
  --threads 1 \
  --storePositions --storeDocvectors --storeRaw

2023-10-09 16:02:49,941 INFO  [main] index.IndexCollection (IndexCollection.java:645) - Setting log level to INFO
2023-10-09 16:02:49,945 INFO  [main] index.IndexCollection (IndexCollection.java:648) - Starting indexer...
2023-10-09 16:02:49,945 INFO  [main] index.IndexCollection (IndexCollection.java:649) - ============ Loading Parameters ============
2023-10-09 16:02:49,946 INFO  [main] index.IndexCollection (IndexCollection.java:650) - DocumentCollection path: tests/resources/sample_collection_jsonl
2023-10-09 16:02:49,948 INFO  [main] index.IndexCollection (IndexCollection.java:651) - CollectionClass: JsonCollection
2023-10-09 16:02:49,949 INFO  [main] index.IndexCollection (IndexCollection.java:652) - Generator: DefaultLuceneDocumentGenerator
2023-10-09 16:02:49,949 INFO  [main] index.IndexCollection (IndexCollection.java:653) - Threads: 1
2023-10-09 16:02:49,950 INFO  [main] index.IndexCollection (IndexCollection.java:654) - Language: en
2023-10-09 16:02:49,950 INFO  [main] index

In [ ]:
!python -m pyserini.search.lucene \
  --threads 16 \
  --batch-size 512 \
  --index wikipedia-dpr-100w \
  --topics dpr-trivia-test \
  --output run.odqa.BM25-k1_0.9_b_0.4.dpr-trivia-test.hits-100.txt \
  --bm25 --k1 0.9 --b 0.4 \
  --hits 100

Attempting to initialize pre-built index wikipedia-dpr-100w.
Unrecognized index name wikipedia-dpr-100w
